# Using C-RADIO with your FiftyOne dataset

First, download a dataset. Let's use the `quickstart`:

In [1]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/INQUIRE_Rerank",
    max_samples=1000
    )

fiftyone.yml:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Loading dataset
Importing samples...
 100% |███████████████| 1000/1000 [29.6ms elapsed, 0s remaining, 33.8K samples/s]      


100%|██████████| 10/10 [00:37<00:00,  3.73s/it]


Next, you register the zoo model source:


In [2]:
import fiftyone.zoo as foz

foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/NVLabs_CRADIOV3"
    )

Finally, instantiate the model. Let's start with computing embeddings.

Note: Refer to the [README](https://github.com/harpreetsahota204/NVLabs_CRADIOV3/blob/main/README.md) for available model checkpoints.

In [ ]:
radio_embeddings_model = foz.load_zoo_model(
    "nv_labs/c-radio_v3-h",
    feature_format="NCHW", # you can also pass NLC here
)

You can compute embeddings as follows:

In [5]:
dataset.compute_embeddings(
    model=radio_embeddings_model,
    embeddings_field="radio_embeddings",
)

 100% |███████████████| 1000/1000 [53.3s elapsed, 0s remaining, 20.3 samples/s]      


Once you have your embeddings, you can compute the visualization to visualize in the FiftyOne App:

In [6]:
import fiftyone.brain as fob

results = fob.compute_visualization(
    dataset,
    method="umap",  # "umap", "tsne", "pca", etc
    brain_key="radio_viz",
    embeddings="radio_embeddings"
)

Generating visualization...


/home/harpreet/miniconda3/envs/fo_develop/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


UMAP( verbose=True)
Wed Jun 18 16:59:05 2025 Construct fuzzy simplicial set
Wed Jun 18 16:59:06 2025 Finding Nearest Neighbors
Wed Jun 18 16:59:10 2025 Finished Nearest Neighbor Search
Wed Jun 18 16:59:11 2025 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Wed Jun 18 16:59:13 2025 Finished embedding


You can also build a similarity index over the embeddings to find similar samples in your dataset:

In [7]:
import fiftyone.brain as fob

results = fob.compute_similarity(
    dataset,
    backend="sklearn",  # "sklearn", "qdrant", "redis", etc
    brain_key="radio_sim",
    embeddings="radio_embeddings"
)

With your computed embeddings you can also perform other embeddings based workflows such as computing uniqueness values:

In [8]:
import fiftyone.brain as fob

fob.compute_uniqueness(
    dataset,
    uniqueness_field="radio_uniqueness",
    similarity_index="radio_sim"
    )

Retrieving embeddings from similarity index...
Computing uniqueness...
Uniqueness computation complete


You can also compute representativeness scores:

In [9]:
import fiftyone.brain as fob

fob.compute_representativeness(
    dataset,
    representativeness_field="radio_representativeness",
    similarity_index="radio_sim"
    )

Retrieving embeddings from similarity index...
Computing representativeness...
Computing clusters for 1000 embeddings; this may take awhile...
Representativeness computation complete


### Computing Spatial Features

You can also compute spatial features. To use this feature you need to set `output_type="spatial"`, additionally spatial features only supports `feature_format="NCHW"`.

You can choose to do some Gaussian smoothing if you'd like, just set `apply_smoothing=True` and choose a value for `smoothing_sigma`.

In [ ]:
radio_spatial_model = foz.load_zoo_model(
    "nv_labs/c-radio_v3-h",
    output_type="spatial",
    apply_smoothing=True, # if you want smoothing
    smoothing_sigma=0.51, # how much smoothing you want to apply
    feature_format="NCHW" #this is the required for the heatmap
) 

Notice that we are using the `apply_model` method here, as we are not computing 1D embeddings like above.

In [11]:
dataset.apply_model(
    radio_spatial_model,
    "radio_spatial_features"
)

 100% |███████████████| 1000/1000 [1.8m elapsed, 0s remaining, 10.5 samples/s]      


You can view your results in the app like so:

In [ ]:
fo.launch_app(dataset)

# ![CRADIO FiftyOne Demo](cradio-fiftyone.gif)